Importing the necessary libraries

In [51]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib

#import warnings
#warnings.filterwarnings("ignore")

import gym
import tensorflow as tf
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Sequential
from collections import deque

Creating a custom map of 4x4,

S: starting point (SAFE)

F: frozen surface (SAFE)

H: Hole in the surface (DEATH)

G: Goal, where the frisbee is located (SAFE)


In [52]:
custom_map = [
    'SFFF',
    'FHFH',
    'FFFH',
    'HFFG'
]

Defining the environemnt along with the training parameters. (Made slippery to False due to ease of implementation)

In [53]:
env = gym.make("FrozenLake-v1", desc=custom_map, is_slippery=False)

train_episodes=4000
test_episodes=100
max_steps=300

state_size = env.observation_space.n # 16 (According to the 4x4 Map)
action_size = env.action_space.n # 4 (According to the 4x4 Map)

batch_size=32
learning_rate=0.001

In [54]:
class Agent:
    def __init__(self, state_size, action_size, learning_rate):
        self.memory = deque(maxlen=2500)
        self.learning_rate=learning_rate
        self.epsilon=1
        self.max_eps=1
        self.min_eps=0.01
        self.eps_decay = 0.001/3
        self.gamma=0.9
        self.state_size= state_size
        self.action_size= action_size
        self.epsilon_lst=[]
        self.model = self.buildmodel()

    def buildmodel(self):
        model=Sequential()
        model.add(Dense(10, input_dim=self.state_size, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def add_memory(self, new_state, reward, done, state, action):
        self.memory.append((new_state, reward, done, state, action))

    def action(self, state):
        if np.random.rand() > self.epsilon:
            return np.random.randint(0,4)
        return np.argmax(self.model.predict(state))

    def pred(self, state):
        return np.argmax(self.model.predict(state))

    def replay(self,batch_size):
        minibatch=random.sample(self.memory, batch_size)
        for new_state, reward, done, state, action in minibatch:
            target= reward
            if not done:
                target=reward + self.gamma* np.amax(self.model.predict(new_state))
            target_f= self.model.predict(state)
            target_f[0][action]= target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.min_eps:
            self.epsilon=(self.max_eps - self.min_eps) * np.exp(-self.eps_decay*episode) + self.min_eps

        self.epsilon_lst.append(self.epsilon)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [55]:
agent=Agent(state_size, action_size, learning_rate)

In [56]:
reward_lst=[]
for episode in range(train_episodes):
    state= env.reset()[0]
    state_arr=np.zeros(state_size)
    state_arr[state] = 1
    state= np.reshape(state_arr, [1, state_size])
    reward = 0
    done = False
    for t in range(max_steps):
        # env.render()
        action = agent.action(state)
        new_state, reward, done, info = env.step(action)
        new_state_arr = np.zeros(state_size)
        new_state_arr[new_state] = 1
        new_state = np.reshape(new_state_arr, [1, state_size])
        agent.add_memory(new_state, reward, done, state, action)
        state= new_state

        if done:
            print(f'Episode: {episode:4}/{train_episodes} and step: {t:4}. Eps: {float(agent.epsilon):.2}, reward {reward}')
            break

    reward_lst.append(reward)

    if len(agent.memory)> batch_size:
        agent.replay(batch_size)

print(' Train mean % score= ', round(100*np.mean(reward_lst),1))

1/1 [==============================] - 0s 59ms/step


ValueError: too many values to unpack (expected 4)